# Preparation

In [100]:
import networkx as nx
import os
import pandas as pd
import re
from collections import Counter
import seaborn as sns
import sys
import hcuppy, json
from hcuppy.cci import CCIEngine
from hcuppy.ccs import CCSEngine
import numpy as np
import math
import csv
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import itertools
import warnings
import gender_guesser.detector as gender
from Bio import Entrez
from IPython.display import HTML, display
from Support.LoadData import (
    EFO_parent_mapper,
    load_gwas_cat,
    load_pubmed_data, 
    make_timely,
    make_clean_CoR,
    download_cat
)
from Support.PubMed import (
    build_collective,
    build_author,
    build_funder,
    build_abstract,
    build_citation
)
from Support.Analysis import (
    simple_facts,
    ancestry_parser,
    make_meanfemale_andranks,
    make_funders,
    mapped_trait_summary
)
from Support.Figures import (
    gwas_growth,
 #   #choropleth_map,
    wordcloud_figure,
    plot_heatmap,
    plot_bubbles,
    boxswarm_plot
)
from Support.Additional import clean_names
from Support.Ancestry import ancestry_cleaner

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'png'
%matplotlib inline
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Download GWAS catalog data from EBI
Then, let's dynamically grab the three main curated datasets from the GWAS Catalogue EBI website that we will need for our endeavours ('All Associations','All Studies', 'All Ancestries') and the EFO to Parent trait mapping file from their FTP site:

In [2]:
ebi = 0
if ebi == 1:
    output_path = os.path.abspath(os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'Catalogue',
                                        'Raw'))
    ebi_download = 'https://www.ebi.ac.uk/gwas/api/search/downloads/'
    download_cat(output_path, ebi_download)

## Link the GWAS catalog data to PubMed
Lets link the PUBMEDID variable to the PUBMED API and get a series of datasets from that using the support functions written in PubMed.py. Note: collective corresponds mostly consortia and study groups.

In [3]:
pubmed = 0
if pubmed == 1: 
    Cat_Studies, Cat_Ancestry, Cat_Ancestry_groupedbyN, Cat_Full = load_gwas_cat()
    id_list = set(Cat_Studies['PUBMEDID'].astype(str).tolist())
    Entrez.email = 'your@email.com'
    papers = Entrez.read(Entrez.efetch(db='pubmed',retmode='xml', id=','.join(id_list)))
    build_collective(papers)
    build_author(papers)
    build_funder(papers)
    build_abstract(papers)
    #build_citation(id_list,Entrez.email)

## Loading the Data into Memory


In [4]:
# load the data
Cat_Studies, Cat_Ancestry, Cat_Ancestry_groupedbyN, Cat_Full = load_gwas_cat()

The"load_gwas_cat()" function loads four datasets:
* **"Cat_Studies"**: each observation is a studied disease/trait, therefore, one publication may appear multiple times if it has involved multiple disease/trait studies.<br> **INFO:** the citation format, sample (inital, replication), disease/trait, study id. <br>
Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'PLATFORM [SNPS PASSING QC]',
       'ASSOCIATION COUNT', 'MAPPED_TRAIT', 'MAPPED_TRAIT_URI',
       'STUDY ACCESSION', 'GENOTYPING TECHNOLOGY'],
      dtype='object')
      
      
* **"Cat_Ancestry"**: each observation is a study-ancenstry pair. if a study incorperates multiple ancestries, then it will appear more than once so as to record all the recruited ancestries. <br> **INFO:** study id, detailed sample info(initial, replication, ancestry, size, country of origin, country of recruitment). <br>
Index(['STUDY ACCESSION', 'PUBMEDID', 'FIRST AUTHOR', 'DATE',
       'INITIAL SAMPLE DESCRIPTION', 'REPLICATION SAMPLE DESCRIPTION', 'STAGE',
       'N', 'BROAD ANCESTRAL', 'COUNTRY OF ORIGIN', 'COUNTRY OF RECRUITMENT',
       'ADDITONAL ANCESTRY DESCRIPTION', 'Broader', 'Dates'],
      dtype='object') 
      
      
* **"Cat_Full"**: each observation is a SNP-trait pair. if a study focuses on one disease/trait, it finds more than one SNP is associated with that disease/trait, then it should record each SNP in this file for that diseaes/trait in that study. <br>
**INFO:** citation format, diseae/trait, sample info, SNP info. <br>
Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'MERGED', 'SNP_ID_CURRENT', 'CONTEXT', 'INTERGENIC',
       'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG', 'P-VALUE (TEXT)',
       'OR or BETA', '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV'],
      dtype='object')

# Ancestry

<font color = "blue"> Mainly using the "Cat_Ancestry" data set" </font>

We use the 'Broad Ancestral Category' field and aggregate from 135 combinations of 17 ancestries to 7 'broader ancestral categories' to calculate a comparable measure to Popejoy and Fullerton.
<br>
<br>
By way of a very simple example: “19,546 British ancestry individuals from 6863 families.” will get cleaned to two seperate fields: “19,546” and “British” which can then be used for further downstream analysis. A slightly more complex example: “2,798 European ancestry individuals, 228 French Canadian founder population individuals” will correspond to two entries of 2798 and 228 in the new ‘Cleaned N’ type variable, corresponding to ‘European’ and ‘French Canadian’ in the ‘Cleaned Ancestry’ type variable respectively

## Classify the detailed ancestry in each study into 7 broad ancestry categories.
Check that our 'Broader' dictionary is up to date?
<font color = 'blue'> In the above, we merge the "Cat_Ancestry" file with the "dict_replacer_broad" to classify the detailed ancestry into several broad categries of races. Lets first now do a quick check that our 'Broader' dictionary is up to date: </font>

In [5]:
# load the ancestry data set:
Cat_Ancestry.head()

,STUDY ACCESSION,PUBMEDID,FIRST AUTHOR,DATE,INITIAL SAMPLE DESCRIPTION,REPLICATION SAMPLE DESCRIPTION,STAGE,N,BROAD ANCESTRAL,COUNTRY OF ORIGIN,COUNTRY OF RECRUITMENT,ADDITONAL ANCESTRY DESCRIPTION,Broader,Dates
7814,GCST000001,15761122,Klein RJ,2005-03-10,"96 European ancestry cases, 50 European ancest...",NaN,initial,146,European,NR,NR,NaN,European,2005-03-10
7808,GCST000002,16252231,Maraganore DM,2005-09-09,"381 European ancestry cases, 363 European ance...","269 European ancestry cases, 272 European ance...",initial,744,European,NR,U.S.,NaN,European,2005-09-09
7809,GCST000002,16252231,Maraganore DM,2005-09-09,"381 European ancestry cases, 363 European ance...","269 European ancestry cases, 272 European ance...",initial,141,Other,NR,U.S.,NaN,Other/Mixed,2005-09-09
7810,GCST000002,16252231,Maraganore DM,2005-09-09,"381 European ancestry cases, 363 European ance...","269 European ancestry cases, 272 European ance...",initial,1,Asian unspecified,NR,U.S.,NaN,Asian,2005-09-09
7811,GCST000002,16252231,Maraganore DM,2005-09-09,"381 European ancestry cases, 363 European ance...","269 European ancestry cases, 272 European ance...",replication,541,European,NR,U.S.,NaN,European,2005-09-09


In [6]:
if len(Cat_Ancestry[Cat_Ancestry['Broader'].isnull()]) > 0:
    print('Perhaps we need to update the dictionary for new terms? Something like these ones:\n' +
         '\n'.join(Cat_Ancestry[Cat_Ancestry['Broader'].isnull()]['BROAD ANCESTRAL'].unique()))
else:
    print('Nice! Looks like our Broad to Broader dictionary is up to date!')

Nice! Looks like our Broad to Broader dictionary is up to date!


## Aggregated Pattern: Check the sample in the initial and replication stages:

In [7]:
## The Initial Stage:
Broad_Ancestral_Full_initial = pd.DataFrame(
    Cat_Ancestry[Cat_Ancestry.STAGE ==
                 'initial'].groupby(['Broader'])['N'].sum())
Broad_Ancestral_Full_initial.rename(
    columns={'N': 'N (Initial)'}, inplace=True)

#Broad_Ancestral_Full_initial.head(10)
# 7 broad race categories


## The Replication Stage:
Broad_Ancestral_Full_replication = pd.DataFrame(
    Cat_Ancestry[Cat_Ancestry.STAGE ==
                 'replication'].groupby(['Broader'])['N'].sum())
Broad_Ancestral_Full_replication.rename(
    columns={'N': 'N (Replication)'}, inplace=True)

#Broad_Ancestral_Full_replication.head(10)



<font color = 'blue' size = 4pt> Merge the initial and replication sample into a total one. </font>

In [8]:
Broad_Ancestral_Full = pd.merge(Broad_Ancestral_Full_initial,
                                Broad_Ancestral_Full_replication,
                                left_index=True, right_index=True)
Broad_Ancestral_Full['Total'] = Broad_Ancestral_Full['N (Initial)'] + \
    Broad_Ancestral_Full['N (Replication)']
Broad_Ancestral_Full['% Discovery'] = (
    Broad_Ancestral_Full['N (Initial)'] /
    Broad_Ancestral_Full['N (Initial)'].sum()) * 100
Broad_Ancestral_Full['% Replication'] = (
    Broad_Ancestral_Full['N (Replication)'] /
    Broad_Ancestral_Full['N (Replication)'].sum()) * 100
Broad_Ancestral_Full['% Total'] = (Broad_Ancestral_Full['Total'] /
                                   Broad_Ancestral_Full['Total'].sum()) * 100
Broad_Ancestral_Full.to_csv(os.path.abspath(
                            os.path.join('__file__',
                                         '../..',
                                         'tables',
                                         'Broad_Ancestral_Full.csv')))
Broad_Ancestral_Full

,N (Initial),N (Replication),Total,% Discovery,% Replication,% Total
Broader,,,,,,
African,1428501,387754,1816255,0.484830,0.422394,0.469998
African Am./Caribbean,4002921,1207839,5210760,1.358583,1.315742,1.348406
Asian,15314535,12634906,27949441,5.197720,13.763655,7.232571
European,249236131,63907946,313144077,84.590208,69.617213,81.033350
Hispanic/Latin American,3075425,1203640,4279065,1.043793,1.311168,1.107308
In Part Not Recorded,19211756,11513163,30724919,6.520429,12.541701,7.950791
Other/Mixed,2370189,943810,3313999,0.804437,1.028126,0.857575


<font color = 'blue' size = 4pt> Drop the 'in part not recorded' rows (i.e. where the Broad Ancestral Category contains at least one NR):
    </font>

In [9]:
Broad_Ancestral_NoNR = Broad_Ancestral_Full[['N (Initial)',
                                             'N (Replication)',
                                             'Total']]
Broad_Ancestral_NoNR = Broad_Ancestral_NoNR.drop('In Part Not Recorded', axis = 0)
Broad_Ancestral_NoNR['Total'] = Broad_Ancestral_NoNR['N (Initial)'] + \
    Broad_Ancestral_NoNR['N (Replication)']
Broad_Ancestral_NoNR['% Discovery'] = (
    Broad_Ancestral_NoNR['N (Initial)'] /
    Broad_Ancestral_NoNR['N (Initial)'].sum()) * 100
Broad_Ancestral_NoNR['% Replication'] = (
    Broad_Ancestral_NoNR['N (Replication)'] /
    Broad_Ancestral_NoNR['N (Replication)'].sum()) * 100
Broad_Ancestral_NoNR['% Total'] = (Broad_Ancestral_NoNR['Total'] /
                                   Broad_Ancestral_NoNR['Total'].sum()) * 100
Broad_Ancestral_NoNR.to_csv(os.path.abspath(
                            os.path.join('__file__', '../..', 'tables',
                                         'Broad_Ancestral_NoNR.csv')))
Broad_Ancestral_NoNR

,N (Initial),N (Replication),Total,% Discovery,% Replication,% Total
Broader,,,,,,
African,1428501,387754,1816255,0.518648,0.482967,0.510595
African Am./Caribbean,4002921,1207839,5210760,1.453347,1.504422,1.464875
Asian,15314535,12634906,27949441,5.560274,15.737392,7.857288
European,249236131,63907946,313144077,90.490582,79.600465,88.032642
Hispanic/Latin American,3075425,1203640,4279065,1.116600,1.499192,1.202952
Other/Mixed,2370189,943810,3313999,0.860549,1.175561,0.931648


<font color = 'blue' size = 4pt> Time Trend  </font> <br>
We can also analyze how these aggregates change over time, and this was a key feature of [Popejoy and Fullerton (2016)](https://www.nature.com/news/genomics-is-failing-on-diversity-1.20759). We can provide a much more granular arguement (merely remove ```'_NoNR'``` from the cell below to undertake an equivlent analysis with rows which contain some in part NR ancestries):

In [10]:
index = [x for x in range(2007, 2019)]
columns = ['European', 'Asian', 'African', 'Hispanic/Latin American',
           'Other/Mixed', 'African Am./Caribbean']
Cat_Ancestry_NoNR = Cat_Ancestry[
    Cat_Ancestry['Broader'] != 'In Part Not Recorded']
Broad_Ancestral_Time_NoNR_PC = pd.DataFrame(index=index, columns=columns)
for year in range(2007, 2019):
    for broaderancestry in Broad_Ancestral_NoNR.index.tolist():
        Broad_Ancestral_Time_NoNR_PC[broaderancestry.strip()][year] =\
            (Cat_Ancestry_NoNR[(
                Cat_Ancestry_NoNR['DATE'].str.contains(str(year))) &
                (Cat_Ancestry_NoNR['Broader'] ==
                 broaderancestry)]['N'].sum() /
             Cat_Ancestry_NoNR[
             (Cat_Ancestry_NoNR['DATE'].str.contains(
                 str(year)))]['N'].sum()) * 100
Broad_Ancestral_Time_NoNR_PC.to_csv(os.path.abspath(
                            os.path.join('__file__', '../..', 'tables',
                                         'Broad_Ancestral_Time_NoNR_PC.csv')))
Broad_Ancestral_Time_NoNR_PC.style

,European,Asian,African,Hispanic/Latin American,Other/Mixed,African Am./Caribbean
2007,95.4688,2.13984,0.00542792,0.715245,1.18391,0.486807
2008,95.2884,2.94597,0,0.00180063,1.21894,0.544872
2009,88.4027,6.96558,0.253076,0.219723,3.29458,0.86432
2010,86.7933,9.91171,0.267259,0.0583916,2.47723,0.49206
2011,78.265,15.7991,0.15655,0.396955,1.70627,3.6762
2012,71.8522,19.0443,0.306187,0.86527,3.30234,4.62965
2013,82.3989,11.6072,0.385825,0.770707,0.61389,4.22351
2014,76.6765,18.0765,0.851494,1.11809,0.949118,2.32834
2015,88.0835,9.04736,0.26423,0.700762,0.49246,1.41172
2016,90.8446,4.47222,0.171732,1.58114,1.01288,1.91738


## Ancestry for Each Study 

We expect to construct a data set with the following format: <br>
**basic info:** <br>
- Study PubMedID year EFO

**ancestry by race and stages:** <br>
- initial: (European Asian, African Hispanic African American Other)
- replication: (European Asian, African Hispanic African American Other) 

**funding per PubmedID:** <br>
- Funding in total
- Funding in US
- Funding in ARRA

In [11]:
# load the ancestry data set:
print(Cat_Ancestry.columns)
Cat_Ancestry.head(2)

Index(['STUDY ACCESSION', 'PUBMEDID', 'FIRST AUTHOR', 'DATE',
       'INITIAL SAMPLE DESCRIPTION', 'REPLICATION SAMPLE DESCRIPTION', 'STAGE',
       'N', 'BROAD ANCESTRAL', 'COUNTRY OF ORIGIN', 'COUNTRY OF RECRUITMENT',
       'ADDITONAL ANCESTRY DESCRIPTION', 'Broader', 'Dates'],
      dtype='object')


,STUDY ACCESSION,PUBMEDID,FIRST AUTHOR,DATE,INITIAL SAMPLE DESCRIPTION,REPLICATION SAMPLE DESCRIPTION,STAGE,N,BROAD ANCESTRAL,COUNTRY OF ORIGIN,COUNTRY OF RECRUITMENT,ADDITONAL ANCESTRY DESCRIPTION,Broader,Dates
7814,GCST000001,15761122,Klein RJ,2005-03-10,"96 European ancestry cases, 50 European ancest...",NaN,initial,146,European,NR,NR,NaN,European,2005-03-10
7808,GCST000002,16252231,Maraganore DM,2005-09-09,"381 European ancestry cases, 363 European ance...","269 European ancestry cases, 272 European ance...",initial,744,European,NR,U.S.,NaN,European,2005-09-09


In [12]:
Ancestry = Cat_Ancestry[['STUDY ACCESSION', 'PUBMEDID', 'DATE', 'INITIAL SAMPLE DESCRIPTION', 'REPLICATION SAMPLE DESCRIPTION',
                         'STAGE', 'N', 'Broader']]

Ancestry = Ancestry.groupby(['STUDY ACCESSION', 'STAGE', 'DATE', 'Broader', 'PUBMEDID'])['N'].sum().reset_index()

Ancestry = Ancestry.pivot_table(
         index = ['STUDY ACCESSION', 'PUBMEDID', 'Broader', 'DATE'],
         columns = 'STAGE',
         values = 'N', 
         fill_value = 0)

Ancestry = Ancestry.unstack(-2, fill_value= 0)
Ancestry.columns = Ancestry.columns.map('_'.join).str.strip('|')

Ancestry['initial_N'] = Ancestry.iloc[:, Ancestry.columns.str.startswith('initial_')].sum(axis = 1)
Ancestry['replication_N'] = Ancestry.iloc[:, Ancestry.columns.str.startswith('replication_')].sum(axis = 1)

Ancestry.reset_index(inplace = True)

print('Is the STUDY-ACCESSION the Unique Identifier in the Acenstry Data?', ~Ancestry.duplicated('STUDY ACCESSION').any())
Ancestry.head()       

Is the STUDY-ACCESSION the Unique Identifier in the Acenstry Data? True


,STUDY ACCESSION,PUBMEDID,DATE,initial_African,initial_African Am./Caribbean,initial_Asian,initial_European,initial_Hispanic/Latin American,initial_In Part Not Recorded,initial_Other/Mixed,replication_African,replication_African Am./Caribbean,replication_Asian,replication_European,replication_Hispanic/Latin American,replication_In Part Not Recorded,replication_Other/Mixed,initial_N,replication_N
0,GCST000001,15761122,2005-03-10,0,0,0,146,0,0,0,0,0,0,0,0,0,0,146,0
1,GCST000002,16252231,2005-09-09,0,0,1,744,0,0,141,0,0,1,541,0,0,122,886,664
2,GCST000003,16614226,2006-04-14,0,0,0,0,0,694,0,0,866,0,10527,0,229,0,694,11622
3,GCST000004,16648850,2006-04-30,0,0,0,200,0,0,0,0,0,0,4851,0,0,0,200,4851
4,GCST000005,17052657,2006-09-28,0,0,0,537,0,0,0,0,0,0,0,0,0,0,537,0


## Merge Funding info data for each article.

In [13]:
Funding = pd.read_csv(os.path.abspath(
                           os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'PUBMED',
                                        'Pubmed_FunderInfo.csv')),
                           header=0, sep=',', encoding='utf-8',
                           index_col=False)
Funding.head()

,PUBMEDID,Agency,GrantCountry,GrantID
0,24029427,NIDDK NIH HHS,United States,U01DK057303
1,24029427,NIDDK NIH HHS,United States,U01 DK057304
2,24029427,Medical Research Council,United Kingdom,MR/K003364/1
3,24029427,Wellcome Trust,United Kingdom,WT090355/A/09/Z
4,24029427,NCRR NIH HHS,United States,M01-RR-01346


In [14]:
Funding['Funded_US'] = (Funding['GrantCountry'] == 'United States').astype(int)
Funding['Funded'] = (Funding['GrantCountry'] != '').astype(int)
Funding['Funded_NIH'] = (Funding['Agency'].str.contains('(NIH)'))
Funding['Funded_AHRQ'] = (Funding['Agency'].str.contains('(AHRQ)'))
Funding['Funded_HHS'] = (Funding['Agency'].str.contains('(HHS)'))
Funding['Funded_ARRA'] = ((Funding['Funded_NIH'] == True) | (Funding['Funded_HHS'] == True) | (Funding['Funded_AHRQ'] == True)).astype(int)

Funding.loc[Funding["Funded_NIH"].apply(math.isnan), 'Funded_NIH'] = 0 
Funding.loc[Funding["Funded_HHS"].apply(math.isnan), 'Funded_HHS'] = 0 
Funding.loc[Funding["Funded_AHRQ"].apply(math.isnan), 'Funded_AHRQ'] = 0 

Funding.replace([True, False], [1,0], inplace = True)
Funding = Funding.groupby('PUBMEDID').sum().reset_index()
print('Is PubMedId the unique identifier in the funding data?', ~Funding.duplicated('PUBMEDID').any())
Funding.head()

Is PubMedId the unique identifier in the funding data? True


,PUBMEDID,Funded_US,Funded,Funded_NIH,Funded_AHRQ,Funded_HHS,Funded_ARRA
0,15761122,6,6,6,0,6,6
1,16252231,8,8,8,0,8,8
2,16614226,11,11,11,0,11,11
3,16648850,3,3,3,0,3,3
4,17052657,1,2,1,0,1,1


In [15]:
Ancestry = pd.merge(Ancestry, Funding, on = 'PUBMEDID',how = 'left',  
                               indicator = 'Funding_merge', validate = 'm:1')

Ancestry.loc[Ancestry["Funded_US"].apply(math.isnan), 'Funded_US'] = 0 
Ancestry.loc[Ancestry["Funded"].apply(math.isnan), 'Funded'] = 0 
Ancestry.loc[Ancestry["Funded_NIH"].apply(math.isnan), 'Funded_NIH'] = 0 
Ancestry.loc[Ancestry["Funded_HHS"].apply(math.isnan), 'Funded_HHS'] = 0 
Ancestry.loc[Ancestry["Funded_AHRQ"].apply(math.isnan), 'Funded_AHRQ'] = 0 
Ancestry.loc[Ancestry["Funded_ARRA"].apply(math.isnan), 'Funded_ARRA'] = 0 

Ancestry.replace({'Funding_merge': {'both':'funding_info_y', 'left_only': 'funding_info_n'}}, inplace = True)

Ancestry.head()


,STUDY ACCESSION,PUBMEDID,DATE,initial_African,initial_African Am./Caribbean,initial_Asian,initial_European,initial_Hispanic/Latin American,initial_In Part Not Recorded,initial_Other/Mixed,...,replication_Other/Mixed,initial_N,replication_N,Funded_US,Funded,Funded_NIH,Funded_AHRQ,Funded_HHS,Funded_ARRA,Funding_merge
0,GCST000001,15761122,2005-03-10,0,0,0,146,0,0,0,...,0,146,0,6.0,6.0,6.0,0.0,6.0,6.0,funding_info_y
1,GCST000002,16252231,2005-09-09,0,0,1,744,0,0,141,...,122,886,664,8.0,8.0,8.0,0.0,8.0,8.0,funding_info_y
2,GCST000003,16614226,2006-04-14,0,0,0,0,0,694,0,...,0,694,11622,11.0,11.0,11.0,0.0,11.0,11.0,funding_info_y
3,GCST000004,16648850,2006-04-30,0,0,0,200,0,0,0,...,0,200,4851,3.0,3.0,3.0,0.0,3.0,3.0,funding_info_y
4,GCST000005,17052657,2006-09-28,0,0,0,537,0,0,0,...,0,537,0,1.0,2.0,1.0,0.0,1.0,1.0,funding_info_y


# Ancestry (By Disease/Trait)

<font color = "blue"> mainly using the "cat_ancstry", "EFO-ICD", and "ICD-CCS" data file </font>
<br>
<br>
In this section, we will match each study (*NOT Publication*) to a specific CCS code through the EFO term (disease/trait) explicitly mentioned in that study. Then we are able to calculate the <font color = "red"> diversity by disease. </font>
<br>
The "EFO-ICD" dictionary is downloaded from https://www.ebi.ac.uk/spot/oxo/search, saved in "data/support" folder
* efo2icd10-mappings_dis1.csv
* efo2icd10-mappings_dis2.csv
* efo2icd10-mappings_dis3.csv <br>
<font color = 'blue'>The distance tells you how many times we have seen this mapping and the distance is a how many hops across other mappings you need to go to find this mapping. Distance 1 is a direct mapping, the greater the distance the less likely it is that a mapping holds true </font>
<br>

The python package "[hcuppy](https://pypi.org/project/hcuppy/)" helps to convert the ICD code to the CCS code.

In [16]:
EFO2ICD10 = pd.read_csv(os.path.abspath(
                           os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'Support',
                                        'efo2icd10-mappings_dis3.csv')),
                           header=0, sep=',', encoding='utf-8',
                           index_col=False)
#EFO2ICD10.head()

# rename the column title
EFO2ICD10.rename(columns = {'curie_id': 'EFO', 'mapped_curie': 'ICD10'}, inplace = True)

print("The shape of the EFO2ICD10 Data:", EFO2ICD10.shape)

EFO2ICD10.head()

The shape of the EFO2ICD10 Data: (7000, 7)


,EFO,label,ICD10,mapped_label,mapping_source_prefix,mapping_target_prefix,distance
0,EFO:0009470,soft tissue disease,ICD10CM:M73,NaN,EFO,ICD10CM,1
1,EFO:0009470,soft tissue disease,ICD10CM:M75,NaN,EFO,ICD10CM,1
2,EFO:0009470,soft tissue disease,ICD10CM:M70,NaN,EFO,ICD10CM,1
3,EFO:0009470,soft tissue disease,ICD10CM:M79,NaN,EFO,ICD10CM,1
4,EFO:0003865,kidney neoplasm,ICD10CM:C63.2,Malignant neoplasm of scrotum,"EFO,MONDO",ICD10CM,3


### To assign each study a corresponding CCS code, we need to translate the EFO term in each study to ICD10, and then from ICD10 to CCS code.

(1) Create a EFO to CCS dictionary (via ICD10) 
* We first extract the <font color = "blue"> EFO ID </font> from the <font color = "blue"> EFO </font> column in the **EFO2ICD10** file. The pattern is that: "EFO:0009470" --> '0009470'.
* Then we match the above **EFO2ICD10** file to the <font color = "blue"> CCS </font> code using the <font color = "blue"> hcuppy </font> package.

(2) match the "CAT_Ancestry" file to the EFO-CCS dictionary

In [17]:
# extract the EFO id for the match in the future.
def find_efo(string):
   # return str(re.findall(r'(\d+)', string)[0])
     return re.sub(r'EFO:', '', string)

EFO2ICD10['EFO'] = EFO2ICD10['EFO'].apply(find_efo)

def find_icd10(string):
    return re.sub(r'ICD10CM:', '', string)
EFO2ICD10['ICD10'] = EFO2ICD10['ICD10'].apply(find_icd10)

#EFO2ICD10.reindex(columns = ['EFO_ID' , 'EFO'] + EFO2ICD10.columns.tolist()[1:-1])
EFO2ICD10.head(10)

,EFO,label,ICD10,mapped_label,mapping_source_prefix,mapping_target_prefix,distance
0,0009470,soft tissue disease,M73,NaN,EFO,ICD10CM,1
1,0009470,soft tissue disease,M75,NaN,EFO,ICD10CM,1
2,0009470,soft tissue disease,M70,NaN,EFO,ICD10CM,1
3,0009470,soft tissue disease,M79,NaN,EFO,ICD10CM,1
4,0003865,kidney neoplasm,C63.2,Malignant neoplasm of scrotum,"EFO,MONDO",ICD10CM,3
5,0003865,kidney neoplasm,C58,Malignant neoplasm of placenta,"EFO,MONDO",ICD10CM,3
6,0003865,kidney neoplasm,C56,NaN,"EFO,MONDO",ICD10CM,3
7,0003865,kidney neoplasm,C73,Malignant neoplasm of thyroid gland,"EFO,MONDO,Orphanet",ICD10CM,3
8,0003865,kidney neoplasm,C64,NaN,EFO,ICD10CM,1
9,0003865,kidney neoplasm,D23,NaN,"EFO,Orphanet",ICD10CM,3


In [18]:
EFO2CCS = EFO2ICD10
ce = CCSEngine(mode="dx")

def find_ccs(string):
    out = ce.get_ccs(string)
    result = json.loads(json.dumps(out, indent = 2))
    return result['ccs']

EFO2CCS['CCS'] = EFO2CCS['ICD10'].apply(find_ccs)

In [19]:
print('In this EFO-CCS dictionary, we have ' + str(EFO2CCS.shape[0]) + ' ICD10 items. Among them, ' + 
       str(len(EFO2CCS[EFO2CCS['CCS'] == 'na'])) + ' are not matched, ' + str(len(EFO2CCS[EFO2CCS['CCS'] != 'na'])) +  
      ' are matched to a CCS code. ' + 'To dive deep, the ICD10 items without a match all have missing (NaN) mapped label.')

print('\nNow one EFO item may be assigned multiple CCS code. So we may need to address this as well.')

EFO2CCS.head(10)

In this EFO-CCS dictionary, we have 7000 ICD10 items. Among them, 2969 are not matched, 4031 are matched to a CCS code. To dive deep, the ICD10 items without a match all have missing (NaN) mapped label.

Now one EFO item may be assigned multiple CCS code. So we may need to address this as well.


,EFO,label,ICD10,mapped_label,mapping_source_prefix,mapping_target_prefix,distance,CCS
0,0009470,soft tissue disease,M73,NaN,EFO,ICD10CM,1,na
1,0009470,soft tissue disease,M75,NaN,EFO,ICD10CM,1,na
2,0009470,soft tissue disease,M70,NaN,EFO,ICD10CM,1,na
3,0009470,soft tissue disease,M79,NaN,EFO,ICD10CM,1,na
4,0003865,kidney neoplasm,C63.2,Malignant neoplasm of scrotum,"EFO,MONDO",ICD10CM,3,31
5,0003865,kidney neoplasm,C58,Malignant neoplasm of placenta,"EFO,MONDO",ICD10CM,3,28
6,0003865,kidney neoplasm,C56,NaN,"EFO,MONDO",ICD10CM,3,na
7,0003865,kidney neoplasm,C73,Malignant neoplasm of thyroid gland,"EFO,MONDO,Orphanet",ICD10CM,3,36
8,0003865,kidney neoplasm,C64,NaN,EFO,ICD10CM,1,na
9,0003865,kidney neoplasm,D23,NaN,"EFO,Orphanet",ICD10CM,3,na


<font color = 'blue'> Tranform the EFO2CCS dictionary into wide format, thus, EFO is able to be an unique identifer. </font>

In [20]:
EFO2CCS.sort_values('EFO').head(20)

,EFO,label,ICD10,mapped_label,mapping_source_prefix,mapping_target_prefix,distance,CCS
5216,0000094,B-cell acute lymphoblastic leukemia,C83.70,"Burkitt lymphoma, unspecified site","EFO,BAO,CLO,DOID",ICD10CM,3,38
5213,0000094,B-cell acute lymphoblastic leukemia,C83.5,NaN,"EFO,MONDO",ICD10CM,3,na
5214,0000094,B-cell acute lymphoblastic leukemia,C83.7,NaN,"EFO,BAO,CLO,DOID",ICD10CM,3,na
5215,0000094,B-cell acute lymphoblastic leukemia,C91.0,NaN,"EFO,MONDO",ICD10CM,3,na
5570,0000095,chronic lymphocytic leukemia,C91.10,Chronic lymphocytic leukemia of B-cell type no...,EFO,ICD10CM,1,39
5569,0000095,chronic lymphocytic leukemia,C91.1,NaN,EFO,ICD10CM,1,na
2332,0000096,neoplasm of mature B-cells,C82.0,NaN,"EFO,MONDO,Orphanet",ICD10CM,3,na
2328,0000096,neoplasm of mature B-cells,C82.1,NaN,"EFO,MONDO,Orphanet",ICD10CM,3,na
2327,0000096,neoplasm of mature B-cells,C82.7,NaN,"EFO,MONDO,Orphanet",ICD10CM,3,na
2326,0000096,neoplasm of mature B-cells,C82.3,NaN,"EFO,MONDO,Orphanet",ICD10CM,3,na


In [21]:
EFO2CCS_long = EFO2CCS[['EFO', 'CCS']].drop_duplicates()
EFO2CCS_long['No'] = 1 
EFO2CCS_long['CCS_LINE'] = EFO2CCS_long.groupby('EFO')['No'].cumsum()
EFO2CCS_long = EFO2CCS_long.pivot(index='EFO', columns='CCS_LINE', values='CCS')
#EFO2CCS_long.columns = EFO2CCS_long.columns.droplevel(0)
#EFO2CCS_long.columns = ['CCS_'.join(col).strip() for col in EFO2CCS_long.columns.values]
EFO2CCS_long.sort_values('EFO').head()

CCS_LINE,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
EFO,,,,,,,,,,,,,,,,
0000094,na,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000095,na,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000096,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000174,na,21,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000178,13,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
EFO2CCS_long.reset_index(inplace=True)
EFO2CCS_long.head()

CCS_LINE,EFO,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0000094,na,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000095,na,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000096,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000174,na,21,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000178,13,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
EFO2CCS_long.reset_index(drop =True)
EFO2CCS_long = EFO2CCS_long.add_prefix('CCS')
EFO2CCS_long= EFO2CCS_long.rename(columns = {'CCSEFO':'EFO'})
EFO2CCS_long.head()

CCS_LINE,EFO,CCS1,CCS2,CCS3,CCS4,CCS5,CCS6,CCS7,CCS8,CCS9,CCS10,CCS11,CCS12,CCS13,CCS14,CCS15,CCS16
0,0000094,na,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000095,na,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000096,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000174,na,21,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000178,13,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge the EFO2CCS dictionary to the GWAS catalog?

<font color = "red">Basic Idea: </font> 
* Match the "Cat_Ancestry" and "Cat_Studies" by the "STUDY ACCESSION", then the Cat_Ancestry will have the *EFO (Diseae/Trait)* label info.
* Match the "EFO2CCS" and "Cat_Ancestry" by the "EFO term?"

Check the unique identifier below before merging the "Cat_Ancestry" and "Cat_Studies" on "STUDY ACCESSION"

<code>
Cat_Ancestry.head(1)
Cat_Studies.head(1)
</code>    

In [24]:
# check the unique identifier in the CAT_STUDIES data

print('Now let\'s check if the study-accession is an unique identifier in the cat_studies data:')
if Cat_Studies.duplicated(['STUDY ACCESSION']).any() == False:
    print('  Wow, that is to say, in the Cat_Study data, the study-accession is an unique identifer in the data.' + '\n'
          '  So we can use the Cat_Study as an dictionary to map the study-accession in the cat-ancestry data to the EFO terms')
else:
    print(' In the Cat_Study data, the study-accession is NOT an unique identifer in the data.' + '\n'
          ' This may cause a problem when merge the cat_studies data to the cat_ancestry data.')

Now let's check if the study-accession is an unique identifier in the cat_studies data:
  Wow, that is to say, in the Cat_Study data, the study-accession is an unique identifer in the data.
  So we can use the Cat_Study as an dictionary to map the study-accession in the cat-ancestry data to the EFO terms


In [25]:
# check the unique identifier in the CAT_ANCESTRY data

print('Now let\'s check if the study-accession is an unique identifier in the Broad_Ancestryal_Study data:')
if Ancestry.duplicated(['STUDY ACCESSION']).any() == False:
    print('  Wow, that is to say, in the Broad_Ancestral_Ancestry data, the study-accession is an unique identifer in the data.' + '\n'
          '  So we can use 1:1 merge with the Broad_Ancestral_Studies data on study-accession.')
else:
    print(' In the Cat_Ancestry data, the study-accession is NOT an unique identifer in the data.' + '\n'
          ' This means, when merging with the Broad_Ancestral_Studies data on study-accession, we need to use the m:1 merge.')

Now let's check if the study-accession is an unique identifier in the Broad_Ancestryal_Study data:
  Wow, that is to say, in the Broad_Ancestral_Ancestry data, the study-accession is an unique identifer in the data.
  So we can use 1:1 merge with the Broad_Ancestral_Studies data on study-accession.


<font color='blue'>Now Let's merge the "Cat_Ancestry" with the "Cat_Studies" on the "STUDY-ACCESSION", the merge validation is 1:1 merge </font>. After this action, we should expect that each item in the "Cat_Ancestry" should have a EFO Term which can be used to merge with the EFO2CCS dictionary.

In [26]:
Ancestry = pd.merge(Ancestry, Cat_Studies, on = 'STUDY ACCESSION', 
                                      how = 'left', validate = '1:1', indicator = 'str')

In [27]:
print('In total there are ' + str(Ancestry.shape[0]) + ' Study-Trait-wise observations in the Broad_Ancestral_Study data.'  + '\n'
      + 'Check the merge outcomes: \n')
if Counter(Ancestry['str'].tolist())['both'] == Ancestry.shape[0]:
    print('Great! All observations in the Broad_Ancestral_Study data have been assigned to and EFO term.')
else:
    print('Opps!' + str(Connter(Ancestry['str'].tolist())['left-only']) + ' of the' + str(Ancestry.shape[0]) + '\n' 
          + 'observations have been assigned to an EFO term.')

Ancestry.head(1)

In total there are 7788 Study-Trait-wise observations in the Broad_Ancestral_Study data.
Check the merge outcomes: 

Great! All observations in the Broad_Ancestral_Study data have been assigned to and EFO term.


,STUDY ACCESSION,PUBMEDID_x,DATE_x,initial_African,initial_African Am./Caribbean,initial_Asian,initial_European,initial_Hispanic/Latin American,initial_In Part Not Recorded,initial_Other/Mixed,...,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,GENOTYPING TECHNOLOGY,str
0,GCST000001,15761122,2005-03-10,0,0,0,146,0,0,0,...,Complement factor H polymorphism in age-relate...,Age-related macular degeneration,"96 European ancestry cases, 50 European ancest...",N/A,Affymetrix [103611],1,age-related macular degeneration,http://www.ebi.ac.uk/efo/EFO_0001365,Genome-wide genotyping array,both


In [28]:
def find_efo_type2(string):
    find = re.findall(r'EFO_\d+', string)
    if find != []:
        find = re.findall(r'EFO_\d+', string)
        find= ','.join(x for x in find)
    if find == []:
        find = None
    return find    

Ancestry['Multiple_EFO'] = Ancestry['MAPPED_TRAIT_URI'].apply(find_efo_type2)

In [29]:
print('Be careful! One study may have multiple mapped-trait-uri, thus, in the EFO column, there may be a list of more than one "EFO-xxxxx" \
      So, we may need to split the EFO column into multiple columns each one of which should contain only one mapped EFO term.')
Ancestry.head(100)

Be careful! One study may have multiple mapped-trait-uri, thus, in the EFO column, there may be a list of more than one "EFO-xxxxx"       So, we may need to split the EFO column into multiple columns each one of which should contain only one mapped EFO term.


,STUDY ACCESSION,PUBMEDID_x,DATE_x,initial_African,initial_African Am./Caribbean,initial_Asian,initial_European,initial_Hispanic/Latin American,initial_In Part Not Recorded,initial_Other/Mixed,...,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,GENOTYPING TECHNOLOGY,str,Multiple_EFO
0,GCST000001,15761122,2005-03-10,0,0,0,146,0,0,0,...,Age-related macular degeneration,"96 European ancestry cases, 50 European ancest...",N/A,Affymetrix [103611],1,age-related macular degeneration,http://www.ebi.ac.uk/efo/EFO_0001365,Genome-wide genotyping array,both,EFO_0001365
1,GCST000002,16252231,2005-09-09,0,0,1,744,0,0,141,...,Parkinson's disease,"381 European ancestry cases, 363 European ance...","269 European ancestry cases, 272 European ance...",Perlegen [198345],1,Parkinson's disease,http://www.ebi.ac.uk/efo/EFO_0002508,Genome-wide genotyping array,both,EFO_0002508
2,GCST000003,16614226,2006-04-14,0,0,0,0,0,694,0,...,Obesity,694 individuals from 288 families,"3,213 European ancestry obese cases, 6,210 Eur...",Affymetrix [86604],0,obesity,http://www.ebi.ac.uk/efo/EFO_0001073,Genome-wide genotyping array,both,EFO_0001073
3,GCST000004,16648850,2006-04-30,0,0,0,200,0,0,0,...,QT interval,100 European ancestry > 445ms female individua...,200 European ancestry > 85th pct female indivi...,Affymetrix [88500],1,QT interval,http://www.ebi.ac.uk/efo/EFO_0004682,Genome-wide genotyping array,both,EFO_0004682
4,GCST000005,17052657,2006-09-28,0,0,0,537,0,0,0,...,Parkinson's disease,"267 European ancestry cases, 270 European ance...",N/A,Illumina [408803],3,Parkinson's disease,http://www.ebi.ac.uk/efo/EFO_0002508,Genome-wide genotyping array,both,EFO_0002508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GCST000096,17903295,2007-09-19,0,0,0,0,0,1345,0,...,Aging traits,"1,345 individuals from 330 families",N/A,Affymetrix [70897],10,"age at menopause, aging, exercise test, age at...","http://www.ebi.ac.uk/efo/EFO_0004704, http://p...",Genome-wide genotyping array,both,"EFO_0004704,EFO_0004328,EFO_0005056"
96,GCST000097,17903303,2007-09-19,0,0,0,0,0,678,0,...,Coronary artery calcification,Up to 678 individuals,N/A,Affymetrix [70897],3,coronary artery calcification,http://www.ebi.ac.uk/efo/EFO_0004723,Genome-wide genotyping array,both,EFO_0004723
97,GCST000098,17903297,2007-09-19,0,0,0,694,0,0,0,...,Cognitive test performance,694 European ancestry individuals,N/A,Affymetrix [70897],14,neuropsychological test,http://www.ebi.ac.uk/efo/EFO_0003926,Genome-wide genotyping array,both,EFO_0003926
98,GCST000099,17903307,2007-09-19,0,0,0,0,0,1222,0,...,Mean forced vital capacity from 2 exams,"Up to 1,222 individuals",N/A,Affymetrix [70897],2,vital capacity,http://www.ebi.ac.uk/efo/EFO_0004312,Genome-wide genotyping array,both,EFO_0004312


In [30]:
Ancestry = Ancestry.join(Ancestry['Multiple_EFO'].str.split(',', expand = True).add_prefix('EFO'))
Ancestry.head()

,STUDY ACCESSION,PUBMEDID_x,DATE_x,initial_African,initial_African Am./Caribbean,initial_Asian,initial_European,initial_Hispanic/Latin American,initial_In Part Not Recorded,initial_Other/Mixed,...,EFO47,EFO48,EFO49,EFO50,EFO51,EFO52,EFO53,EFO54,EFO55,EFO56
0,GCST000001,15761122,2005-03-10,0,0,0,146,0,0,0,...,None,None,None,None,None,None,None,None,None,None
1,GCST000002,16252231,2005-09-09,0,0,1,744,0,0,141,...,None,None,None,None,None,None,None,None,None,None
2,GCST000003,16614226,2006-04-14,0,0,0,0,0,694,0,...,None,None,None,None,None,None,None,None,None,None
3,GCST000004,16648850,2006-04-30,0,0,0,200,0,0,0,...,None,None,None,None,None,None,None,None,None,None
4,GCST000005,17052657,2006-09-28,0,0,0,537,0,0,0,...,None,None,None,None,None,None,None,None,None,None


<font color = 'blue' size='4pt'> Reshape the Ancestral_Study data from wide format into long format. Each observation is a Study-EFO pair-wise. </font>

In [31]:
# FIRST, create unique id for each observation for reshape.
Ancestry['ID'] = Ancestry.index
Ancestry.head(1)

,STUDY ACCESSION,PUBMEDID_x,DATE_x,initial_African,initial_African Am./Caribbean,initial_Asian,initial_European,initial_Hispanic/Latin American,initial_In Part Not Recorded,initial_Other/Mixed,...,EFO48,EFO49,EFO50,EFO51,EFO52,EFO53,EFO54,EFO55,EFO56,ID
0,GCST000001,15761122,2005-03-10,0,0,0,146,0,0,0,...,None,None,None,None,None,None,None,None,None,0


In [32]:
# Second, reshape from wide to long, id = 'ID', EFO_LINE = 1,2,3.4...56
Ancestry = pd.wide_to_long(Ancestry, stubnames= 'EFO', i = 'ID',  j ='EFO_LINE')
# drop the observations when EFO is none
Ancestry = Ancestry[Ancestry['EFO'].notnull()]
print('The long-format Ancestral_Study_long data has ' + str(Ancestry.shape[0]) + ' Observations with non-missing EFO terms.')
print('However, look at the EFO columns, the format is not suitable for merging to the EFO2CCS dictionary. So we address this now.')
print('We need to transform: EFO_0001365 --> 0001365')
Ancestry['EFO'] = Ancestry['EFO'].str.replace('EFO_', '')
Ancestry.head(100)

The long-format Ancestral_Study_long data has 9927 Observations with non-missing EFO terms.
However, look at the EFO columns, the format is not suitable for merging to the EFO2CCS dictionary. So we address this now.
We need to transform: EFO_0001365 --> 0001365


,,Funded_AHRQ,Funded,initial_African Am./Caribbean,DATE_x,initial_In Part Not Recorded,Multiple_EFO,initial_European,initial_Asian,replication_European,initial_Hispanic/Latin American,...,INITIAL SAMPLE SIZE,replication_African,DATE ADDED TO CATALOG,replication_In Part Not Recorded,replication_African Am./Caribbean,Funding_merge,PUBMEDID_x,DATE_y,JOURNAL,EFO
ID,EFO_LINE,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,6.0,0,2005-03-10,0,EFO_0001365,146,0,0,0,...,"96 European ancestry cases, 50 European ancest...",0,2008-06-16,0,0,funding_info_y,15761122,2005-03-10,Science,0001365
1,0,0.0,8.0,0,2005-09-09,0,EFO_0002508,744,1,541,0,...,"381 European ancestry cases, 363 European ance...",0,2008-06-16,0,0,funding_info_y,16252231,2005-09-09,Am J Hum Genet,0002508
2,0,0.0,11.0,0,2006-04-14,694,EFO_0001073,0,0,10527,0,...,694 individuals from 288 families,0,2009-02-14,229,866,funding_info_y,16614226,2006-04-14,Science,0001073
3,0,0.0,3.0,0,2006-04-30,0,EFO_0004682,200,0,4851,0,...,100 European ancestry > 445ms female individua...,0,2008-06-16,0,0,funding_info_y,16648850,2006-04-30,Nat Genet,0004682
4,0,0.0,2.0,0,2006-09-28,0,EFO_0002508,537,0,0,0,...,"267 European ancestry cases, 270 European ance...",0,2008-06-16,0,0,funding_info_y,17052657,2006-09-28,Lancet Neurol,0002508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0.0,9.0,0,2007-09-19,0,EFO_0003926,694,0,0,0,...,694 European ancestry individuals,0,2008-09-10,0,0,funding_info_y,17903297,2007-09-19,BMC Med Genet,0003926
98,0,0.0,3.0,0,2007-09-19,1222,EFO_0004312,0,0,0,0,...,"Up to 1,222 individuals",0,2009-09-28,0,0,funding_info_y,17903307,2007-09-19,BMC Med Genet,0004312
99,0,0.0,3.0,0,2007-09-19,883,EFO_0004748,0,0,0,0,...,883 individuals,0,2008-09-02,0,0,funding_info_y,17903292,2007-09-19,BMC Med Genet,0004748


In [33]:
print('The EFO2CCS dictionary has ' + str(EFO2CCS_long.shape[0]) + ' observations.')
EFO2CCS_long.head()

The EFO2CCS dictionary has 2136 observations.


CCS_LINE,EFO,CCS1,CCS2,CCS3,CCS4,CCS5,CCS6,CCS7,CCS8,CCS9,CCS10,CCS11,CCS12,CCS13,CCS14,CCS15,CCS16
0,0000094,na,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000095,na,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000096,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000174,na,21,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000178,13,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
Ancestry = pd.merge(Ancestry, EFO2CCS_long, on = 'EFO',how = 'left',  indicator = 'CCS_merge', validate = 'm:1')
print('Each Study-EFO obserbation may be matched to more than one CCS code. This is currently a wide format, For further analysis, \
      it should be reshape to long format, each observation is a Study-EFO-CCS pair.')
Ancestry.head()

Each Study-EFO obserbation may be matched to more than one CCS code. This is currently a wide format, For further analysis,       it should be reshape to long format, each observation is a Study-EFO-CCS pair.


,Funded_AHRQ,Funded,initial_African Am./Caribbean,DATE_x,initial_In Part Not Recorded,Multiple_EFO,initial_European,initial_Asian,replication_European,initial_Hispanic/Latin American,...,CCS8,CCS9,CCS10,CCS11,CCS12,CCS13,CCS14,CCS15,CCS16,CCS_merge
0,0.0,6.0,0,2005-03-10,0,EFO_0001365,146,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,0.0,8.0,0,2005-09-09,0,EFO_0002508,744,1,541,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,0.0,11.0,0,2006-04-14,694,EFO_0001073,0,0,10527,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,0.0,3.0,0,2006-04-30,0,EFO_0004682,200,0,4851,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0.0,2.0,0,2006-09-28,0,EFO_0002508,537,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [35]:
Ancestry['ID'] = Ancestry.index
Ancestry = pd.wide_to_long(Ancestry, stubnames= 'CCS', i = 'ID',  j ='CCS_LINE')
Ancestry = Ancestry.reset_index(level = 1, drop = True)

In [36]:
Ancestry = Ancestry[Ancestry['CCS'].notnull()]
Ancestry = Ancestry[Ancestry['CCS'] != 'na']
print('In the Ancestral_Study_CCS data, each observation is a Study-EFO-CCS pair. That is to say, a study may be assigned to \
      multiple EFO terms, and a typical EFO term may be matched to multiple CCS code. Therefore, in the Ancestral_Study_CCS \
      data, a study may appear more than once, and meanwhile, a EFO term may appear more than once.')
Ancestry.head()

In the Ancestral_Study_CCS data, each observation is a Study-EFO-CCS pair. That is to say, a study may be assigned to       multiple EFO terms, and a typical EFO term may be matched to multiple CCS code. Therefore, in the Ancestral_Study_CCS       data, a study may appear more than once, and meanwhile, a EFO term may appear more than once.


,replication_Hispanic/Latin American,Funded_AHRQ,Funded,initial_African Am./Caribbean,DATE_x,initial_N,REPLICATION SAMPLE SIZE,initial_In Part Not Recorded,Multiple_EFO,initial_Other/Mixed,...,ASSOCIATION COUNT,Funded_US,DATE_y,Funded_NIH,EFO,LINK,JOURNAL,str,Funded_ARRA,CCS
ID,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,6.0,0,2005-03-10,146,N/A,0,EFO_0001365,0,...,1,6.0,2005-03-10,6.0,0001365,www.ncbi.nlm.nih.gov/pubmed/15761122,Science,both,6.0,95
1,0,0.0,8.0,0,2005-09-09,886,"269 European ancestry cases, 272 European ance...",0,EFO_0002508,141,...,1,8.0,2005-09-09,8.0,0002508,www.ncbi.nlm.nih.gov/pubmed/16252231,Am J Hum Genet,both,8.0,79
2,0,0.0,11.0,0,2006-04-14,694,"3,213 European ancestry obese cases, 6,210 Eur...",694,EFO_0001073,0,...,0,11.0,2006-04-14,11.0,0001073,www.ncbi.nlm.nih.gov/pubmed/16614226,Science,both,11.0,58
4,0,0.0,2.0,0,2006-09-28,537,N/A,0,EFO_0002508,0,...,3,1.0,2006-09-28,1.0,0002508,www.ncbi.nlm.nih.gov/pubmed/17052657,Lancet Neurol,both,1.0,79
5,0,0.0,0.0,0,2006-10-19,226,N/A,0,EFO_0001365,0,...,1,0.0,2006-10-19,0.0,0001365,www.ncbi.nlm.nih.gov/pubmed/17053108,Science,both,0.0,95


In [37]:
Ancestry.columns

Index(['replication_Hispanic/Latin American', 'Funded_AHRQ', 'Funded',
       'initial_African Am./Caribbean', 'DATE_x', 'initial_N',
       'REPLICATION SAMPLE SIZE', 'initial_In Part Not Recorded',
       'Multiple_EFO', 'initial_Other/Mixed', 'PUBMEDID_y', 'initial_African',
       'initial_European', 'initial_Asian', 'replication_European',
       'initial_Hispanic/Latin American', 'FIRST AUTHOR', 'STUDY',
       'GENOTYPING TECHNOLOGY', 'Funded_HHS', 'replication_N',
       'INITIAL SAMPLE SIZE', 'DISEASE/TRAIT', 'replication_African',
       'CCS_merge', 'DATE ADDED TO CATALOG',
       'replication_In Part Not Recorded', 'PLATFORM [SNPS PASSING QC]',
       'MAPPED_TRAIT', 'replication_Other/Mixed',
       'replication_African Am./Caribbean', 'STUDY ACCESSION', 'Funding_merge',
       'replication_Asian', 'PUBMEDID_x', 'MAPPED_TRAIT_URI',
       'ASSOCIATION COUNT', 'Funded_US', 'DATE_y', 'Funded_NIH', 'EFO', 'LINK',
       'JOURNAL', 'str', 'Funded_ARRA', 'CCS'],
      dtype='ob

# Merge CCS-Disease Dictionary

In [38]:
CCS_Disease = pd.read_excel(os.path.abspath(
                           os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'MEPS',
                                        'Support',
                                        'CCS_Disease_Dictionary.xlsx')),
                           header=0, encoding='utf-8',
                           index_col=False)
CCS_Disease['CCS'] = CCS_Disease['CCS'].astype(str)
CCS_Disease.head()

,CCS,Disease
0,1,Infectious diseases
1,2,Infectious diseases
2,3,Infectious diseases
3,4,Infectious diseases
4,5,Infectious diseases


In [39]:
Ancestry = pd.merge(Ancestry, CCS_Disease, on = 'CCS', 
                                      how = 'left', validate = 'm:1')


In [40]:
Ancestry.columns

Index(['replication_Hispanic/Latin American', 'Funded_AHRQ', 'Funded',
       'initial_African Am./Caribbean', 'DATE_x', 'initial_N',
       'REPLICATION SAMPLE SIZE', 'initial_In Part Not Recorded',
       'Multiple_EFO', 'initial_Other/Mixed', 'PUBMEDID_y', 'initial_African',
       'initial_European', 'initial_Asian', 'replication_European',
       'initial_Hispanic/Latin American', 'FIRST AUTHOR', 'STUDY',
       'GENOTYPING TECHNOLOGY', 'Funded_HHS', 'replication_N',
       'INITIAL SAMPLE SIZE', 'DISEASE/TRAIT', 'replication_African',
       'CCS_merge', 'DATE ADDED TO CATALOG',
       'replication_In Part Not Recorded', 'PLATFORM [SNPS PASSING QC]',
       'MAPPED_TRAIT', 'replication_Other/Mixed',
       'replication_African Am./Caribbean', 'STUDY ACCESSION', 'Funding_merge',
       'replication_Asian', 'PUBMEDID_x', 'MAPPED_TRAIT_URI',
       'ASSOCIATION COUNT', 'Funded_US', 'DATE_y', 'Funded_NIH', 'EFO', 'LINK',
       'JOURNAL', 'str', 'Funded_ARRA', 'CCS', 'Disease'],
    

In [41]:

Ancestry.rename(columns = {'initial_African Am./Caribbean':'ini_African_American', 
                            'initial_Hispanic/Latin American':'ini_Hispanic',
                            'initial_Asian': 'ini_Asian',
                            'initial_European': 'ini_European',
                            'initial_Other/Mixed': 'ini_Mixed',
                            'initial_In Part Not Recorded': 'ini_NR',
                            'initial_African': 'ini_African',
                            'replication_African Am./Caribbean':'rep_African_American', 
                            'replication_Hispanic/Latin American':'rep_Hispanic',
                            'replication_Asian': 'rep_Asian',
                            'replication_European': 'rep_European',
                            'replication_Other/Mixed': 'rep_Mixed',
                            'replication_In Part Not Recorded': 'rep_NR',
                            'replication_African': 'rep_African',
                            'initial_N':'ini_N',
                            'replication_N': 'rep_N'}, inplace = True)



Ancestry['N_African'] = Ancestry['ini_African'] + Ancestry['rep_African']
Ancestry['N_African_American'] = Ancestry['ini_African_American'] + Ancestry['rep_African_American'] 
Ancestry['N_Asian'] = Ancestry['ini_Asian'] + Ancestry['rep_Asian'] 
Ancestry['N_European'] = Ancestry['ini_European'] + Ancestry['rep_European']       
Ancestry['N_Hispanic'] = Ancestry['ini_Hispanic'] + Ancestry['rep_Hispanic']  
Ancestry['N_Mixed'] = Ancestry['ini_Mixed']  + Ancestry['rep_Mixed']
Ancestry['N_NR'] = Ancestry['ini_NR']  + Ancestry['rep_NR']

Ancestry['N'] = Ancestry['ini_N'] + Ancestry['rep_N']  


In [42]:
Ancestry.to_csv(os.path.abspath(os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'Catalogue',
                                        'Synthetic',
                                        'Ancestral_Study_CCS.csv')))

---

## Find the ARRA Supported Studies (2009-2010)

### Loading the ARRA grant list

In [172]:
ARRA = pd.read_excel(os.path.abspath(os.path.join('__file__',
                                                         '../..',
                                                         'data',
                                                         'Support',
                                                         'NIH ARRA Grant List',
                                                         'ARRA-projects.xls')), 
                             sheet_name = "ARRA", header=0, encoding='utf-8',
                             index_col=False)

ARRA.columns = ARRA.columns.str.strip()

In [173]:
ARRA.shape

(21712, 25)

In [174]:
ARRA['Project Number'] = ARRA['Project Number'].str.strip()

### Loading the GWAS study grant info list

In [242]:
GWAS_Grant = pd.read_csv(os.path.abspath(os.path.join('__file__',
                                                         '../..',
                                                         'data',
                                                         'PUBMED',
                                                         'Pubmed_FunderInfo.csv')),
                             header=0, encoding='utf-8',
                             index_col=False, sep =',')

GWAS_Grant.columns = GWAS_Grant.columns.str.strip()
GWAS_Grant['GrantID'] = GWAS_Grant['GrantID'].str.replace(' ', '')
GWAS_Grant['GrantID'] = GWAS_Grant['GrantID'].fillna('NOT REPORTED')
GWAS_Grant.head()

,PUBMEDID,Agency,GrantCountry,GrantID
0,24029427,NIDDK NIH HHS,United States,U01DK057303
1,24029427,NIDDK NIH HHS,United States,U01DK057304
2,24029427,Medical Research Council,United Kingdom,MR/K003364/1
3,24029427,Wellcome Trust,United Kingdom,WT090355/A/09/Z
4,24029427,NCRR NIH HHS,United States,M01-RR-01346


Example of **fuzzywuzzy**:
<code>
fuzzResult = process.extract("R01CA082659", choice, scorer = fuzz.partial_ratio, limit = 5) 
fuzzResult
[('3R01CA082659-11S1', 100, 11437),
 ('3R01CA018689-33S1', 82, 779),
 ('3R01CA018689-33S2', 82, 780),
 ('2R01CA085419-10', 82, 2332),
 ('5R01CA085419-11', 82, 2352)]
</code>

In [243]:
choice = ARRA['Project Number']
GWAS_Grant = GWAS_Grant[GWAS_Grant['GrantCountry'] == 'United States'][['GrantID']].drop_duplicates()
GWAS_Grant.shape

(10848, 1)

In [244]:
%%time
for index, row in GWAS_Grant.iterrows():
    fuzzResult = process.extract((row['GrantID']), choice , scorer = fuzz.partial_ratio, limit = 3)
    GWAS_Grant.loc[index, 'ARRA_MatchedGrant1'] = fuzzResult[0][0]
    GWAS_Grant.loc[index, 'ARRA_MatchedScore1'] = fuzzResult[0][1]
    GWAS_Grant.loc[index, 'ARRA_MatchedGrant2'] = fuzzResult[1][0]
    GWAS_Grant.loc[index, 'ARRA_MatchedScore2'] = fuzzResult[1][1]
    GWAS_Grant.loc[index, 'ARRA_MatchedGrant3'] = fuzzResult[2][0]
    GWAS_Grant.loc[index, 'ARRA_MatchedScore3'] = fuzzResult[2][1]
    
GWAS_Grant.to_csv(os.path.abspath(os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'Catalogue',
                                        'Synthetic',
                                        'fuzz.csv')))

Wall time: 8h 28min 25s


## Match the GWAS Grant Info data with the ARRA Grant Detailed info via the Matched GrantID

In [246]:
GWAS_Grant.head()

,GrantID,ARRA_MatchedGrant1,ARRA_MatchedScore1,ARRA_MatchedGrant2,ARRA_MatchedScore2,ARRA_MatchedGrant3,ARRA_MatchedScore3
0,U01DK057303,3U01DK061730-08S1,82.0,3U01DK061730-09S1,82.0,3U01DK062503-08S1,82.0
1,U01DK057304,3U01DK061734-09S1,82.0,3U01DK072504-05S1,82.0,3U01DK057149-11S1,82.0
4,M01-RR-01346,3R01RR014166-16S1,67.0,3R01RR021746-03S1,67.0,3R01RR023168-03S1,58.0
5,U01DK57304,3U01DK057149-11S1,80.0,3R01DK057030-08S1,80.0,3U01DK057131-11S1,80.0
6,U01DK057298,3U01DK057219-12S1,91.0,3U01DK061728-09S1,82.0,3U01DK072493-04S1,82.0


In [247]:
ARRA.head()

,State,Organization,City,Award Type,Project Number,DUNS Number,Congressional District,Agency Code,Award Notice Issued,FY Total Cost,...,Activity,IC,Serial Number,Support Year,Suffix,Sub #,Principal Investigator,Admin IC,FY,Project End Date
0,CA,"23ANDME, INC.",MOUNTAIN VIEW,Grant,1R43HG005807-01,780119710,14.0,7529,2010-09-05,189844,...,R43,HG,5807,1,NaN,NaN,"MOUNTAIN, JOANNA L.",NHGRI,2010,2012-05-31
1,NC,3-C INSTITUTE FOR SOCIAL DEVELOPMENT,CARY,Grant,1R43RR030780-01,46981549,2.0,7529,2010-07-08,199995,...,R43,RR,30780,1,NaN,NaN,"DEROSIER, MELISSA E.",NCRR,2010,2011-07-07
2,NC,3-C INSTITUTE FOR SOCIAL DEVELOPMENT,CARY,Grant,3R43MH086983-02S1,46981549,2.0,7529,2010-09-27,240698,...,R43,MH,86983,2,S1,NaN,"SWICK, DANIELLE CORI",NIMH,2010,2011-09-26
3,MA,"4S3 BIOSCIENCE, INC.",MEDFORD,Grant,1R21NS066269-01,828286703,7.0,7529,2009-07-20,141092,...,R21,NS,66269,1,NaN,NaN,"ARMSTRONG, DUSTIN",NINDS,2009,2011-07-31
4,MA,"4S3 BIOSCIENCE, INC.",MEDFORD,Grant,5R21NS066269-02,828286703,7.0,7529,2010-06-24,195590,...,R21,NS,66269,2,NaN,NaN,"ARMSTRONG, DUSTIN",NINDS,2010,2013-07-31


### CAUTIOUS: Project Number '3R37DK044083-19S1" is duplicated twice in the ARRA list. It is the only one with duplicates. The info in these two duplicates are the same. So we are safe to drop one.

In [262]:
ARRA = ARRA.drop_duplicates('Project Number')

(21712, 25)

### Merge the ARRA Grant detailed information to the GWAS Grant list.

In [265]:
GWAS_ARRA_Grant = pd.merge(GWAS_Grant, ARRA, left_on ="ARRA_MatchedGrant1", right_on ="Project Number", 
                           validate = "m:1", indicator = "ARRA_merge" )

In [266]:
GWAS_ARRA_Grant.head()

,GrantID,ARRA_MatchedGrant1,ARRA_MatchedScore1,ARRA_MatchedGrant2,ARRA_MatchedScore2,ARRA_MatchedGrant3,ARRA_MatchedScore3,State,Organization,City,...,IC,Serial Number,Support Year,Suffix,Sub #,Principal Investigator,Admin IC,FY,Project End Date,ARRA_merge
0,U01DK057303,3U01DK061730-08S1,82.0,3U01DK061730-09S1,82.0,3U01DK062503-08S1,82.0,MD,JOHNS HOPKINS UNIVERSITY,BALTIMORE,...,DK,61730,8,S1,NaN,"TONASCIA, JAMES A",NIDDK,2010,2011-01-31,both
1,U01DK061730,3U01DK061730-08S1,100.0,3U01DK061730-09S1,100.0,3U01DK061732-08S1,91.0,MD,JOHNS HOPKINS UNIVERSITY,BALTIMORE,...,DK,61730,8,S1,NaN,"TONASCIA, JAMES A",NIDDK,2010,2011-01-31,both
2,U01DK057304,3U01DK061734-09S1,82.0,3U01DK072504-05S1,82.0,3U01DK057149-11S1,82.0,NY,COLUMBIA UNIVERSITY HEALTH SCIENCES,NEW YORK,...,DK,61734,9,S1,NaN,"LAVINE, JOEL EDWARD",NIDDK,2010,2012-01-31,both
3,U01DK061734,3U01DK061734-09S1,100.0,3U01DK066174-08S1,91.0,3U01DK066174-08S2,91.0,NY,COLUMBIA UNIVERSITY HEALTH SCIENCES,NEW YORK,...,DK,61734,9,S1,NaN,"LAVINE, JOEL EDWARD",NIDDK,2010,2012-01-31,both
4,M01-RR-01346,3R01RR014166-16S1,67.0,3R01RR021746-03S1,67.0,3R01RR023168-03S1,58.0,AZ,ARIZONA STATE UNIVERSITY-TEMPE CAMPUS,TEMPE,...,RR,14166,16,S1,NaN,"SMITH, BRIAN H.",NCRR,2009,2011-09-16,both


In [268]:
GWAS_funding = pd.read_csv(os.path.abspath(os.path.join('__file__',
                                                         '../..',
                                                         'data',
                                                         'PUBMED',
                                                         'Pubmed_FunderInfo.csv')),
                             header=0, encoding='utf-8',
                             index_col=False, sep =',')

GWAS_funding.columns = GWAS_funding.columns.str.strip()
GWAS_funding['GrantID'] = GWAS_funding['GrantID'].str.replace(' ', '')
GWAS_funding['GrantID'] = GWAS_funding['GrantID'].fillna('NOT REPORTED')

In [269]:
GWAS_ARRA_Grant = pd.merge(GWAS_funding, GWAS_ARRA_Grant, left_on ="GrantID", right_on ="GrantID", 
                           validate = "m:1", indicator = "funding_merge" )

GWAS_ARRA_Grant.head()

In [279]:
GWAS_ARRA_Grant['IsARRA'] = (GWAS_ARRA_Grant['ARRA_MatchedScore1'] >= 97)
GWAS_ARRA_Grant['IsARRA'].astype(int)

GWAS_ARRA_Grant = GWAS_ARRA_Grant.groupby('PUBMEDID')['IsARRA'].sum()

GWAS_ARRA_Grant = GWAS_ARRA_Grant.reset_index()

In [293]:
Ancestry = pd.merge(Ancestry, GWAS_ARRA_Grant, left_on = 'PUBMEDID_y', right_on = "PUBMEDID", 
                                      how = 'left', validate = 'm:1')

Index(['rep_Hispanic', 'Funded_AHRQ', 'Funded', 'ini_African_American',
       'DATE_x', 'ini_N', 'REPLICATION SAMPLE SIZE', 'ini_NR', 'Multiple_EFO',
       'ini_Mixed', 'PUBMEDID_y', 'ini_African', 'ini_European', 'ini_Asian',
       'rep_European', 'ini_Hispanic', 'FIRST AUTHOR', 'STUDY',
       'GENOTYPING TECHNOLOGY', 'Funded_HHS', 'rep_N', 'INITIAL SAMPLE SIZE',
       'DISEASE/TRAIT', 'rep_African', 'CCS_merge', 'DATE ADDED TO CATALOG',
       'rep_NR', 'PLATFORM [SNPS PASSING QC]', 'MAPPED_TRAIT', 'rep_Mixed',
       'rep_African_American', 'STUDY ACCESSION', 'Funding_merge', 'rep_Asian',
       'PUBMEDID_x', 'MAPPED_TRAIT_URI', 'ASSOCIATION COUNT', 'Funded_US',
       'DATE_y', 'Funded_NIH', 'EFO', 'LINK', 'JOURNAL', 'str', 'Funded_ARRA',
       'CCS', 'Disease', 'N_African', 'N_African_American', 'N_Asian',
       'N_European', 'N_Hispanic', 'N_Mixed', 'N_NR', 'N'],
      dtype='object')

In [296]:
Ancestry.to_csv(os.path.abspath(os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'Catalogue',
                                        'Synthetic',
                                        'Ancestral_Study_CCS.csv')))